In [18]:
import pandas as pd

# load data
df = pd.read_csv('processed_data/combined_madden_ratings.csv')

/var/folders/wc/fbq6xjkd6fggx6lcm7rwdnfc0000gn/T/ipykernel_41849/1458016042.py:4: DtypeWarning: Columns (5,7,8,11,65,66,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('processed_data/combined_madden_ratings.csv')


In [29]:
cols = df.columns
df.query('madden > 13 & madden < 25').groupby('madden').apply(lambda x: x.isna().sum()).apply(lambda x: x > 100)

,madden,team,parsed_team,first_name,last_name,full_name,position,jersey_number,salary,age,...,kick_accuracy,kick_return,stamina,injury,player_handedness,running_style,archetype,tackle,pursuit,morale
madden,,,,,,,,,,,,,,,,,,,,,
14,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,True,True,True,False,False,True
15,False,True,False,False,False,True,False,False,True,True,...,False,False,False,False,True,True,True,False,False,True
16,False,True,False,False,False,True,False,False,True,True,...,False,False,False,False,True,True,True,False,False,True
17,False,False,False,False,False,True,False,True,True,True,...,False,False,False,False,True,True,True,False,False,True
18,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,True,True,False,False,True
19,False,False,False,True,True,False,False,False,False,False,...,False,False,False,False,True,True,True,False,False,True
20,False,False,False,True,True,False,False,False,False,False,...,False,False,False,False,True,True,True,False,False,True
21,False,False,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
22,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True


In [30]:
import nfl_data_py as nfl

In [36]:
df = nfl.import_pbp_data([2014], downcast=True, cache=False, alt_path=None)

2014 done.
Downcasting floats.


In [34]:
df = nfl.import_weekly_data([2014])

Downcasting floats.


In [42]:
# Load play-by-play data
pbp_data = nfl.import_pbp_data(years=list(range(2003, 2023)))

# Selecting and mutating columns


2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.


TypeError: first() missing 1 required positional argument: 'offset'

In [54]:
game_data = (
    pbp_data
    # Selecting columns
    .filter(items=[
        'play_id', 'game_id', 'yards_gained', 'old_game_id', 'home_team', 'away_team', 'season_type', 
        'week', 'posteam', 'yardline_100', 'game_date', 'qtr', 'down', 'ydstogo', 'ydsnet', 'play_type', 
        'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'ep', 
        'epa', 'total_home_epa', 'total_away_epa', 'total_home_rush_epa', 'total_away_rush_epa', 
        'total_home_pass_epa', 'total_away_pass_epa', 'third_down_converted', 'third_down_failed', 
        'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'interception', 'fumble_forced', 
        'fumble_not_forced', 'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'pass_touchdown', 'rush_touchdown', 
        'two_point_attempt', 'field_goal_attempt','fumble', 'passing_yards', 'rushing_yards', 'penalty_yards', 
        'penalty_team', 'season', 'start_time', 'play_type_nfl', 'drive_time_of_possession', 'fixed_drive_result', 
        'away_score', 'home_score', 'div_game', 'roof', 'surface', 'temp', 'wind', 'stadium_id', 'game_stadium'
    ])
    # Performing mutations
    .assign(
        home_penalty_yards=lambda x: x['penalty_yards'].where(x['penalty_team'] == x['home_team'], 0),
        away_penalty_yards=lambda x: x['penalty_yards'].where(x['penalty_team'] != x['home_team'], 0),
        pass_yards_gained=lambda x: x['yards_gained'].where(x['play_type'] == 'pass'),
        run_yards_gained=lambda x: x['yards_gained'].where(x['play_type'] == 'run'),
        run_attempt=lambda x: x['play_type'].apply(lambda y: 1 if y == 'run' else 0),
        field_goal_made=lambda x: x['field_goal_result'].apply(lambda y: 1 if y == 'made' else 0),
        extra_point_good=lambda x: x['extra_point_result'].apply(lambda y: 1 if y == 'good' else 0),
        pass_epa=lambda x: x['epa'].where(x['play_type'] == 'pass'),
        rush_epa=lambda x: x['epa'].where(x['play_type'] == 'run')
    )
    # Grouping and summarizing
    .groupby([
        'game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 
        'game_date', 'season', 'away_score', 'home_score', 'div_game', 'roof', 'surface'
    ])
    .agg(
        avg_epa=('epa', 'mean'),
        avg_pass_epa=('pass_epa', 'mean'),
        avg_rush_epa=('rush_epa', 'mean'),
        n_third_down_converted=('third_down_converted', 'sum'),
        n_third_down_failed=('third_down_failed', 'sum'),
        n_fourth_down_converted=('fourth_down_converted', 'sum'),
        n_fourth_down_failed=('fourth_down_failed', 'sum'),
        n_pass_attempts=('pass_attempt', 'sum'),
        n_run_attempts=('run_attempt', 'sum'),
        n_incomplete_pass=('incomplete_pass', 'sum'),
        n_interception=('interception', 'sum'),
        n_fumble_forced=('fumble_forced', 'sum'),
        n_fumble_not_forced=('fumble_not_forced', 'sum'),
        n_qb_hit=('qb_hit', 'sum'),
        n_sacks=('sack', 'sum'),
        n_two_point_attempts=('two_point_attempt', 'sum'),
        n_field_goal_attempts=('field_goal_attempt', 'sum'),
        n_fumbles=('fumble', 'sum'),
        sum_home_penalty_yards=('home_penalty_yards', 'sum'),
        sum_away_penalty_yards=('away_penalty_yards', 'sum'),
        n_pass_touchdown=('pass_touchdown', 'sum'),
        n_rush_touchdown=('rush_touchdown', 'sum'),
        field_goal_success_rate=('field_goal_made', lambda x: x.mean(skipna=True)),
        extra_point_success_rate=('extra_point_good', lambda x: x.mean(skipna=True)),
        avg_pass_yards_gained=('pass_yards_gained', 'mean'),
        avg_run_yards_gained=('run_yards_gained', 'mean'),
        temp=('temp', 'mean'),
        wind=('wind', 'mean')
    )
    .assign(
        fourth_down_attempts=lambda x: x['n_fourth_down_converted'] + x['n_fourth_down_failed'],
        n_attempts=lambda x: x['n_pass_attempts'] + x['n_run_attempts'],
        n_touchdowns=lambda x: x['n_pass_touchdown'] + x['n_rush_touchdown'],
    )
    .reset_index()
)

# Additional processing to calculate rates and percentages
game_data = game_data.assign(
    third_down_conversion_rate=game_data['n_third_down_converted'] / (game_data['n_third_down_converted'] + game_data['n_third_down_failed']),
    fourth_down_conversion_rate=game_data['n_fourth_down_converted'] / game_data['fourth_down_attempts'],
    percentage_pass_attempts=game_data['n_pass_attempts'] / (game_data['n_pass_attempts'] + game_data['n_run_attempts']),
    percentage_run_attempts=game_data['n_run_attempts'] / (game_data['n_pass_attempts'] + game_data['n_run_attempts']),
    percentage_incomplete_pass=game_data['n_incomplete_pass'] / game_data['n_pass_attempts'],
    percentage_interception=game_data['n_interception'] / game_data['n_pass_attempts'],
    percentage_fumbles=game_data['n_fumbles'] / game_data['n_attempts'],
    percentage_qb_hits=game_data['n_qb_hit'] / game_data['n_attempts'],
    percentage_sacks=game_data['n_sacks'] / game_data['n_attempts'],
    pass_touchdown_rate=game_data['n_pass_touchdown'] / game_data['n_touchdowns'],
    offense=lambda x: x['home_team'].where(x['home_team'] == x['posteam'], x['away_team']),
    defense=lambda x: x['away_team'].where(x['home_team'] == x['posteam'], x['home_team']),
    score=lambda x: x['home_score'].where(x['home_team'] == x['posteam'], x['away_score']),
)

In [55]:
game_data.columns

Index(['game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam',
       'game_date', 'season', 'away_score', 'home_score', 'div_game', 'roof',
       'surface', 'avg_epa', 'avg_pass_epa', 'avg_rush_epa',
       'n_third_down_converted', 'n_third_down_failed',
       'n_fourth_down_converted', 'n_fourth_down_failed', 'n_pass_attempts',
       'n_run_attempts', 'n_incomplete_pass', 'n_interception',
       'n_fumble_forced', 'n_fumble_not_forced', 'n_qb_hit', 'n_sacks',
       'n_two_point_attempts', 'n_field_goal_attempts', 'n_fumbles',
       'sum_home_penalty_yards', 'sum_away_penalty_yards', 'n_pass_touchdown',
       'n_rush_touchdown', 'field_goal_success_rate',
       'extra_point_success_rate', 'avg_pass_yards_gained',
       'avg_run_yards_gained', 'temp', 'wind', 'fourth_down_attempts',
       'n_attempts', 'n_touchdowns', 'third_down_conversion_rate',
       'fourth_down_conversion_rate', 'percentage_pass_attempts',
       'percentage_run_attempts', 'percentage_in

In [56]:
game_cols = ['game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam',
             'game_date', 'season', 'away_score', 'home_score', 'div_game', 'roof',
             'surface', 'offense', 'defense', 'offense_score', 'defense_score', 'temp', 'wind']
stat_cols = ['avg_epa', 'avg_pass_epa', 'avg_rush_epa',
            'n_third_down_converted', 'n_third_down_failed',
            'n_fourth_down_converted', 'n_fourth_down_failed', 'n_pass_attempts',
            'n_run_attempts', 'n_incomplete_pass', 'n_interception',
            'n_fumble_forced', 'n_fumble_not_forced', 'n_qb_hit', 'n_sacks',
            'n_two_point_attempts', 'n_field_goal_attempts', 'n_fumbles',
            'sum_home_penalty_yards', 'sum_away_penalty_yards', 'n_pass_touchdown',
            'n_rush_touchdown', 'field_goal_success_rate',
            'extra_point_success_rate', 'avg_pass_yards_gained',
            'avg_run_yards_gained', 'third_down_conversion_rate',
            'fourth_down_attempts',
            'n_attempts', 'n_touchdowns', 'third_down_conversion_rate',
            'fourth_down_conversion_rate', 'percentage_pass_attempts',
            'percentage_run_attempts', 'percentage_incomplete_pass',
            'percentage_interception', 'percentage_fumbles', 'percentage_qb_hits',
            'percentage_sacks', 'pass_touchdown_rate']
offense_cols = ['game_id', 'season', 'week', 'offense', 'offense_score']
defense_cols = ['game_id', 'season', 'week', 'defense', 'defense_score']
             